# The cooperative example

In [1]:
import asyncio
import time

In [2]:
# override print to allow showing the output in one cell as it was printed in a terminal
import os

real_print = print
lines = []
def print(text):
    global lines
    lines.append(text)

In [3]:
async def notify_every(every):
    while keep_running:
        print("Notify!")
        await asyncio.sleep(every)

In [4]:
async def exit_after(exit_after):
    global keep_running
    
    await asyncio.sleep(exit_after)
    keep_running = False

But how can this be transformed into something more friendly?

All that needs to be done is to split our *long* `time.sleep` call into a series of many *shorter* `time.sleep` calls, which in the end add up to the same amount of sleep time. 

A call to `asyncio.sleep` with in minimalist sleep time allows intercepting the 'work' and execute other things in parallel.

The `friendly_blocking_wait_for` function immplements just this.

In [6]:
async def friendly_blocking_wait_for(wait_for):
    print(f"I'm going to sleep right now for {wait_for} seconds.")
    
    # split blocking wait_for into parts
    number_of_parts=10
    sub_wait_for = wait_for / number_of_parts
    
    tic = time.perf_counter()
    for index in range(number_of_parts):
        # That call is still blocking!
        time.sleep(sub_wait_for)
        # This little 'pause' allows asyncio to trigger things in between
        await asyncio.sleep(1e-6)
    toc = time.perf_counter()
    
    print(
        "Hey y'all. That was a good nap! "
        f"Slept for {toc-tic:.2f} seconds. "
        "Did i miss something?"
        )

Let's check if that already helped.

In [7]:
keep_running = True
exec_for = 5
wait_for = 2
_notify_every = .2

tic = time.perf_counter()
_ = await asyncio.gather(
    exit_after(exec_for),
    notify_every(_notify_every),
    friendly_blocking_wait_for(wait_for),
)
toc = time.perf_counter()

print(f"All over execution time was {toc - tic:.2f} seconds.")

In [8]:
real_print("\n".join(lines))
lines = []

Notify!
I'm going to sleep right now for 2 seconds.
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Hey y'all. That was a good nap! Slept for 2.04 seconds. Did i miss something?
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
Notify!
All over execution time was 5.05 seconds.


Much better!

So for every function that has to be interceptable a special implementatio allowing this needs to be available. 

And that's exactly the case. Find *doubles* of almost all blocking Python functions below the [asyncio part of Pythons documentation](https://docs.python.org/3/library/asyncio.html).